# Make an animated GIF with one Big Data Bowl play

This dataset was adapted from animation code in the Kaggle discussion section of the Big Data Bowl 2023.

In [ ]:
!apt-get -qq update
!apt install -y -qq imagemagick 

In [10]:
import pandas as pd 
import seaborn as sns 
sns.set(rc={'axes.facecolor':'mediumseagreen'})
import numpy as np
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='html5')

In [ ]:
ppfScoutingData = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2023/pffScoutingData.csv')
games = pd.read_csv('../input/nfl-big-data-bowl-2023/games.csv')
players = pd.read_csv('../input/nfl-big-data-bowl-2023/players.csv')
plays = pd.read_csv('../input/nfl-big-data-bowl-2023/plays.csv')
week1 = pd.read_csv('../input/nfl-big-data-bowl-2023/week1.csv')
week2 = pd.read_csv('../input/nfl-big-data-bowl-2023/week2.csv')
week3 = pd.read_csv('../input/nfl-big-data-bowl-2023/week3.csv')
week4 = pd.read_csv('../input/nfl-big-data-bowl-2023/week4.csv')
week5 = pd.read_csv('../input/nfl-big-data-bowl-2023/week5.csv')
week6 = pd.read_csv('../input/nfl-big-data-bowl-2023/week6.csv')
week7 = pd.read_csv('../input/nfl-big-data-bowl-2023/week7.csv')
week8 = pd.read_csv('../input/nfl-big-data-bowl-2023/week8.csv')
testing = pd.read_csv('/kaggle/input/anothera/all_weeks_processed1.csv')

In [13]:
#reflect the play because we originally transform all cordinates around y = x
# in order to make them vertical, however we must reflect them to make it not a mirrored image
def reflect(arr):
    lst = []
 
    for i in arr:
        diff = abs(i - 26.65)
        if i - 26.65 > 0:
            lst.append(26.65-diff)
        else:
             lst.append(26.65 + diff)
    return lst
def objective(x, a, b, c):
    return b * x + a * x**2 + c


In [14]:
def animate_play1(one_play):    
    gid = one_play['gameId'].unique()[0]
    pid = one_play['playId'].unique()[0]
  # get line of scrimmage info from the football X location from the  first frame of data 
    los = one_play.loc[(one_play['frameId']==1) & (one_play['team']=='football'), 'x'].values[0]
    fig = plt.figure(figsize=( 8,18))
    ax = fig.gca()
    ax.cla()
    figsize=( 8,18)
    sns.set(rc={'figure.figsize':( 8,18)})
    #field for visualization
    rect = patches.Rectangle((0, 0), 53.3,120, linewidth=0.1,edgecolor='r', facecolor='darkgreen', zorder=0)
    ax.add_patch(rect)
    first_frame = one_play[one_play['frameId']==0]
    first_frame['y'] = reflect(first_frame['y'])
    fig1 = sns.scatterplot(x='y',y='x',data=first_frame, 
                             hue='team',palette = ['brown','blue','yellow'], ax=ax, s=100)
    #plotting hash marks and other field things 
    fig1.legend([]).set_visible(False)
    fig1.plot(
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
        [10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             color='white')   
    for x in range(20, 110, 10):
        numb = x
        if x > 50:
            numb = 120 - x
        fig1.text( 5, x - .97, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation = 270)
        fig1.text( 53.3 - 5,x - .95, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=90)
        fig1.text( 53.3 /2,5, 'ENDZONE',
                 horizontalalignment='center',
                    fontsize=20,  # fontname='Arial',
                     color='white')
        fig1.text( 53.3 /2,115, 'ENDZONE',
                 horizontalalignment='center',
                    fontsize=20,  # fontname='Arial',
                     color='white')
        
        
        hash_range = range(11, 110)
    
    # Endzones
       
        fig1.axis('off')
        for x in hash_range:
            ax.plot( [0.4, 0.7],[x, x], color='white')
            ax.plot( [53.0, 52.5],[x, x], color='white')
            ax.plot( [22.91, 23.57],[x, x], color='white')
            ax.plot( [29.73, 30.39],[x, x], color='white')
            
       
    

    ani = animation.FuncAnimation(fig, get_play_by_frame_updated33, 
                                frames=one_play['frameId'].unique().shape[0],
                                interval=100, repeat=True, 
                                fargs=(ax,los,one_play,))
  
    plt.close
    return ani    

In [8]:
def get_play_by_frame_updated33(fid, ax, los, one_play):
    #temp fix
    print(fid)
    if(fid != 0):
        absolute_yardline_number = one_play['absoluteYardlineNumber'].values[0]
        yards_to_go = one_play['yardsToGo'].values[0]
        play_direction = one_play['playDirection'].values[0]
        first_down_line = absolute_yardline_number + yards_to_go if play_direction == "right" else absolute_yardline_number - yards_to_go  
        ax.cla()
        figsize=( 8,18)
        sns.set(rc={'figure.figsize':( 8,18)})
        rect = patches.Rectangle((0, 0), 53.3,120, linewidth=0.1,edgecolor='r', facecolor='darkgreen', zorder=0)  
        ax.add_patch(rect)
        gid = one_play['gameId'].unique()[0]
        pid = one_play['playId'].unique()[0]
        one_frame = one_play.loc[one_play['frameId']==fid]
        one_frame['y']= reflect(one_frame['y'])
        fig1 = sns.scatterplot(x='y',y='x',data=one_frame, 
                             hue='team',palette = ['brown','blue','yellow'], ax=ax, s=100)
        fig1.set_facecolor("blue")
        fig1.legend([]).set_visible(False)
        fig1.plot(
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
        [10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             color='white')
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            fig1.text( 5, x - .97, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation = 270)
            fig1.text( 53.3 - 5,x - .95, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=90)
            fig1.text( 53.3 /2,5, 'ENDZONE',
                 horizontalalignment='center',
                    fontsize=20,  # fontname='Arial',
                     color='white')
            fig1.text( 53.3 /2,115, 'ENDZONE',
                 horizontalalignment='center',
                    fontsize=20,  # fontname='Arial',
                     color='white') 
        hash_range = range(11, 110)
        fig1.axis('off')
        for x in hash_range:
            ax.plot( [0.4, 0.7],[x, x], color='white')
            ax.plot( [53.0, 52.5],[x, x], color='white')
            ax.plot( [22.91, 23.57],[x, x], color='white')
            ax.plot( [29.73, 30.39],[x, x], color='white')
        #a b and c values for equation of shield/parabola    
        a = one_frame['a_y'].values[0]
        b = one_frame['b'].values[0]
        c = one_frame['c'].values[0]
        is_good_pocket = one_frame['is_good_pocket'].values[0]
        o_line = one_frame[one_frame['officialPosition'].isin(['C', 'T', 'G'])]
        x = (o_line['y'].values)
        y = o_line['x'].values
        x_line = ''
        y_line = ''
        color = 'white'
        if(is_good_pocket!="1"):
            color = 'red'
        first_down_line = absolute_yardline_number + yards_to_go if play_direction == "right" else absolute_yardline_number - yards_to_go
        x_line = np.arange(min(x)-3, max(x)+3, 1)
        y_line = objective(x_line, a, b, c)
   

# Plays used for analysis pictures

In [ ]:
this_game = week1[week1['gameId']==2021090900]
this_play = this_game[this_game['playId']==1587]
one_play =  this_play.merge(testing, on=['gameId','playId','frameId'], how='left')
one_play_updated = one_play.merge(plays,on = ['gameId','playId'], how = 'left').merge(players,on = 'nflId',how='left')

In [ ]:
this_game = week1[week1['gameId']==2021090900]
this_play = this_game[this_game['playId']==4772]
one_play =  this_play.merge(testing, on=['gameId','playId','frameId'], how='left')
one_play_updated = one_play.merge(plays,on = ['gameId','playId'], how = 'left').merge(players,on = 'nflId',how='left')

In [ ]:
this_game = week1[week1['gameId']==2021090900]
this_play = this_game[this_game['playId']==2279]
one_play =  this_play.merge(testing, on=['gameId','playId','frameId'], how='left')
one_play_updated = one_play.merge(plays,on = ['gameId','playId'], how = 'left').merge(players,on = 'nflId',how='left')


In [ ]:
this_game = week8[week8['gameId']==2021110100]
this_play = this_game[this_game['playId']==3362]
one_play =  this_play.merge(testing, on=['gameId','playId','frameId'], how='left')
one_play_updated = one_play.merge(plays,on = ['gameId','playId'], how = 'left').merge(players,on = 'nflId',how='left')


In [ ]:
animate_play1(one_play_updated)